# Web scraping to generate RSS feed for new positions in economics
This application is using three main sources to retrieve information about the new job posts:
1. [NBER](https://www.nber.org/career-resources/research-assistant-positions-not-nber)
2. [Predoc](https://predoc.org/opportunities)
3. [EconJobMarket](https://econjobmarket.org/market)
   
The packeges that are needed are **requests**, **beautifulsoup4**,**MIMEtext**. As a first step we recall them:


In [4]:

import requests
import certifi
from bs4 import BeautifulSoup
import re
import csv
import os
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from IPython.display import Markdown, display
from email.mime.multipart import MIMEMultipart
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv() #just for email and password 

True

In [5]:
CSV_FILE = "jobs.csv" #database saved in simple csv file
PREDOC_URL = "https://predoc.org/opportunities"
NBER_URL = "https://www.nber.org/career-resources/research-assistant-positions-not-nber"
EJM_URL = "https://econjobmarket.org/market"



## Downloading the html
The following functions are downloading the HTML content from the sources and it save it in the foulder sources.
For PREDOC there is a issue with certificate so it is easy to use curl (bash MacOS)

In [6]:
!mkdir -p sources
!curl -L "https://predoc.org/opportunities" -o "sources/predoc.html"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  407k  100  407k    0     0   155k      0  0:00:02  0:00:02 --:--:--  155k 3  407k    3 15550    0     0   7798      0  0:00:53  0:00:01  0:00:52  7798


In [7]:
def download_html(url, filename):
    """
    Downloads the HTML content from the given URL and saves it to the specified filename.
    """
    try:
        response = requests.get(url, verify=certifi.where())
        response.raise_for_status()
        with open(filename, "w", encoding="utf-8") as f:
            f.write(response.text)
        print(f"Downloaded HTML from {url} to {filename}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")

# Ensure the 'sources' folder exists.
os.makedirs("sources", exist_ok=True)

# Download HTML content for each source.
download_html(PREDOC_URL, "sources/predoc.html")
download_html(NBER_URL, "sources/nber.html")
download_html(EJM_URL, "sources/ejm.html")


Error downloading https://predoc.org/opportunities: HTTPSConnectionPool(host='predoc.org', port=443): Max retries exceeded with url: /opportunities (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1010)')))
Downloaded HTML from https://www.nber.org/career-resources/research-assistant-positions-not-nber to sources/nber.html
Downloaded HTML from https://econjobmarket.org/market to sources/ejm.html




## Web Scraping Section 🚀

In this section, we set up our web scraping functionality. Our goal is to **extract job details** from pre-doctoral opportunities pages (in this example, from [predoc.org](https://predoc.org/opportunities)). We assume that the HTML content has already been downloaded and saved locally in the `sources` folder.

### Predoc
What This Section Does:
- **Reads the Local HTML File 📂:**  
  We read the downloaded HTML file (`sources/predoc.html`). If the file isn't found, the code prompts you to download it first.
  
- **Parses the HTML Content 🥣:**  
  Using BeautifulSoup, the code parses the HTML to locate the container that holds the opportunity details.
  
- **Extracts Key Information 🔍:**  
  For each job posting, the function extracts:
  - **Program Title** and **Link** from the `<h2>` element.
  - Additional details (like **sponsor**, **institution**, **fields of research**, and **deadline**) from the "copy" `<div>`.
  
- **Determines the Main Field 🔑:**  
  It combines several text fields and passes them to an auxiliary function (`extract_main_field()`) that determines the primary focus (e.g., Economics, Microeconomics, Finance, etc.).

- **Returns the Data as a List 📤:**  
  Each job is stored as a dictionary, and the function returns a list of these dictionaries.

> **Note:**  
> Make sure to download the HTML file before running the scraper (therefore run the previous chunks).


In [8]:
def scrape_predoc():
    """
    Scrapes the pre-doctoral opportunities page from the local HTML file
    and extracts job details.
    """
    jobs = []
    
    # Attempt to read the local HTML file. 📂
    try:
        with open("sources/predoc.html", "r", encoding="utf-8") as f:
            html = f.read()
    except Exception as e:
        print("Error reading sources/predoc.html. Please download the HTML from predoc before proceeding. 🚫")
        return jobs  # Return an empty list if the file can't be read.
    
    # Parse the HTML content using BeautifulSoup. 🥣
    soup = BeautifulSoup(html, "html.parser")
    
    # Find the container holding the opportunities using a regex on the class name. 🔍
    container = soup.find("div", class_=re.compile("Opportunities"))
    if not container:
        print("No Predoc container found. 😢")
        return jobs
    
    # Loop over each article element within the container. 📝
    articles = container.find_all("article")
    for article in articles:
        job = {}
        job["source"] = "predoc"  # Mark the source as 'predoc'. 🌟
        
        # Extract the title and link from the <h2> element. 🏷️
        h2 = article.find("h2")
        if h2:
            a_tag = h2.find("a")
            if a_tag:
                job["program_title"] = a_tag.get_text(strip=True)
                job["link"] = a_tag.get("href", "").strip()
            else:
                job["program_title"] = "N/A"
                job["link"] = ""
        else:
            job["program_title"] = "N/A"
            job["link"] = ""
        
        # Extract details from the "copy" div. 🗒️
        copy_div = article.find("div", class_="copy")
        if copy_div:
            p = copy_div.find("p")
            if p:
                text = p.get_text(separator=" ", strip=True)
                # Use regex to capture specific fields from the text. 🔍
                researcher_match = re.search(r"Sponsoring Researcher\(s\):\s*(.*?)\s*Sponsoring Institution:", text)
                institution_match = re.search(r"Sponsoring Institution:\s*(.*?)\s*Fields of Research", text)
                fields_match = re.search(r"Fields of Research\s*:\s*(.*?)\s*Deadline:", text)
                deadline_match = re.search(r"Deadline:\s*(.*)", text)
                job["sponsor"] = researcher_match.group(1).strip() if researcher_match else "N/A"
                job["institution"] = institution_match.group(1).strip() if institution_match else "N/A"
                job["fields"] = fields_match.group(1).strip() if fields_match else "N/A"
                job["deadline"] = deadline_match.group(1).strip() if deadline_match else "N/A"
            else:
                job["sponsor"] = "N/A"
                job["institution"] = "N/A"
                job["fields"] = "N/A"
                job["deadline"] = "N/A"
        else:
            job["sponsor"] = "N/A"
            job["institution"] = "N/A"
            job["fields"] = "N/A"
            job["deadline"] = "N/A"
        
        # Add additional fields for consistency. 🛠️
        job["university"] = "N/A"
        job["program_type"] = "N/A"
        job["publication_date"] = "N/A"
        
        # Determine the main field by combining text from various fields. 🔑
        text_to_search = " ".join([job.get("fields", ""), job.get("program_title", ""), job.get("institution", "")])
        job["main_field"] = extract_main_field(text_to_search)
        
        # Append the extracted job details to the jobs list. ✅
        jobs.append(job)
    
    # Return the list of all extracted job details. 📤
    return jobs


# Web Scraping Section for NBER (Local HTML) 🔎

In this section, we extract job details from the locally saved NBER page HTML file. The function follows these steps:

- **📂 Read the Local HTML File:**  
  The function attempts to read `sources/nber.html`. If the file isn't found, it prints an error message and returns an empty list.

- **🥣 Parse HTML with BeautifulSoup:**  
  The HTML content is parsed so we can navigate and extract the data.

- **🔍 Locate the Container:**  
  It finds the `<div>` with class `page-header__intro-inner` that holds the job details.

- **✂️ Skip Header Paragraphs:**  
  The first three `<p>` elements are skipped as they contain header information.

- **📋 Extract Job Details:**  
  For each job posting, the function extracts:
  - Program title  
  - Sponsor  
  - Institution  
  - Fields of research  
  - Job link  
  If any of these details are missing, it defaults to `"N/A"`.

- **🔑 Determine Main Field:**  
  It combines relevant text and uses the helper function `extract_main_field()` (which should be defined elsewhere) to determine the primary research area.

- **✅ Return the Jobs List:**  
  Finally, all extracted job entries are stored in a list and returned.


In [ ]:
def scrape_nber():
    """
    Scrapes the NBER research assistant positions page from a local HTML file
    and extracts job details.
    """
    jobs = []
    
    # Attempt to read the local HTML file. 📂
    try:
        with open("sources/nber.html", "r", encoding="utf-8") as f:
            html = f.read()
    except Exception as e:
        print("Error reading sources/nber.html. Please download the HTML from NBER before proceeding. 🚫")
        return jobs  # Return an empty list if the file can't be read.
    
    # Parse the HTML content using BeautifulSoup. 🥣
    soup = BeautifulSoup(html, "html.parser")
    
    # Find the container holding the job details using its class name. 🔍
    container = soup.find("div", class_="page-header__intro-inner")
    if container:
        # Get all <p> elements inside the container. 📝
        paragraphs = container.find_all("p")
        # Skip the first three header paragraphs. ✂️
        for p in paragraphs[2:]:
            job = {}
            job["source"] = "NBER"  # Mark the source as NBER. 🌟
            parts = p.decode_contents().split("<br>")[0].split("<br/>")


            if len(parts) >= 4:
                job["program_title"]        = parts[0].strip()
                job["sponsor"]              = parts[1].replace("NBER Sponsoring Researcher(s):", "").strip()
                job["institution"]          = parts[2].replace("Institution:", "").strip()
                if len(parts[3].replace("Field(s) of Research:", "").strip().split("&amp"))>1:
                    fields = "".join(field.strip() for field in parts[3].replace("Field(s) of Research:", "").strip().split("&amp"))
                else:
                    fields         = parts[3].replace("Field(s) of Research:", "").strip()
                if len(fields.split(";")) > 1:
                    fields = ", ".join(field.strip() for field in fields.split(";"))

                if len(fields.split(":")) > 1:
                    fields = fields.split(":")[1]
                job["fields"]               =   fields
                job["program_type"]         = extract_program_type(job["program_title"])
                # Combine text fields (program title and university) to determine the main field. 🔑
                text_to_search =  fields
                job["main_field"] = extract_main_field(text_to_search)
                # Extract the job link from the HTML in the last part. 🔗
                link_soup = BeautifulSoup(parts[4], "html.parser")
                a_tag = link_soup.find("a")
                job["link"]                 = a_tag["href"] if a_tag else ""
                job["deadline"]             = "N/A"  # Deadline not provided. ⏰
                job["publication_date"]     = "N/A"
                # Append the extracted job to our list. ✅
                jobs.append(job)
    else:
        print("NBER container not found. 😢")
    
    # Return the list of all extracted job details. 📤
    return jobs
df  = pd.DataFrame(scrape_nber()).head(10).to_markdown(index=False)
display(Markdown(df))

| source   | program_title                                                 | sponsor                       | institution                                                                        | fields                                                                     | program_type        | main_field                | link                                                                                                                                      | deadline   | publication_date   |
|:---------|:--------------------------------------------------------------|:------------------------------|:-----------------------------------------------------------------------------------|:---------------------------------------------------------------------------|:--------------------|:--------------------------|:------------------------------------------------------------------------------------------------------------------------------------------|:-----------|:-------------------|
| nber     | Predoctoral Research Analyst                                  | Josh Rauh                     | Hoover Institution                                                                 | State, Local Economic Policy                                               | PreDoctoral Program | Local Economic Policy     | https://careersearch.stanford.edu/jobs/research-analyst-for-state-and-local-government-initiative-27190                                   | N/A        | N/A                |
| nber     | Postdoctoral Research Fellowship                              | Josh Rauh                     | Hoover Institution                                                                 | State, Local Economic Policy                                               | Post Doc            | Local Economic Policy     | https://applyrtf.hoover.org/                                                                                                              | N/A        | N/A                |
| nber     | Predoctoral Research Associate                                | Paul A. Gompers               | Harvard Business School                                                            | Entrepreneurship, Finance                                                  | PreDoctoral Program | Entrepreneurship, Finance | https://www.dropbox.com/scl/fi/71ook3ulz90ncx67qvf8d/Gompers_predoc_job_posting_2025.pdf?rlkey=nqoh9hutr0gprkm8xyukhrm00&st=9h2t9x7v&dl=0 | N/A        | N/A                |
| nber     | Predoctoral Fellow                                            | Amanda Pallais                | Harvard University                                                                 | Labor Economics                                                            | PreDoctoral Program | Economics                 | https://academicpositions.harvard.edu/postings/14690                                                                                      | N/A        | N/A                |
| nber     | Full-Time Pre-Doctoral Fellow/Research Assistant              | Stephan Heblich               | University of Toronto, Department of Economics and Rotman School of Management     | Applied Economics                                                          | PreDoctoral Program | Economics                 | https://foslab.org/join-our-team/pre-doc-fellowships/pre-doctoral-fellowship-in-applied-economics/                                        | N/A        | N/A                |
| nber     | Pre-doctoral Research Fellows                                 | Gordon Hanson and Dani Rodrik | Harvard Kennedy School                                                             | Regional dimensions of inequality in the United States and other countries | PreDoctoral Program | N/A                       | https://www.hks.harvard.edu/centers/wiener/programs/economy/about/opportunities/pre-doc-fellow-2025                                       | N/A        | N/A                |
| nber     | Research Professional                                         | Michael Greenstone            | University of Chicago, Climate Impact Lab                                          | Environment, Climate                                                       | Research Assistant  | Climate                   | https://job-boards.greenhouse.io/uchicagoepic/jobs/6349570003                                                                             | N/A        | N/A                |
| nber     | Pre-doctoral Fellow                                           | Lisa B. Kahn                  | University of Rochester                                                            | Labor Economics                                                            | PreDoctoral Program | Economics                 | https://docs.google.com/forms/d/e/1FAIpQLSd2VitZ6wmUBL0Qyqiyg-zj-hSOjXIuMVke4XSJI4bLri0VDg/viewform?pli=1                                 | N/A        | N/A                |
| nber     | Monitoring &amp; Evaluation Specialist, EPIC Air Quality Fund | Michael Greenstone            | University of Chicago (but not a direct hire, will work with an EOR)               | Air Quality, Environment, Climate                                          | Research Assistant  | Climate                   | https://epic.uchicago.edu/opportunities/monitoring-evaluation-specialist-epic-air-quality-fund/                                           | N/A        | N/A                |
| nber     | Research Analyst                                              | Dean Karlan, Christopher Udry | Global Poverty Research Lab, Kellogg School of Management, Northwestern University | Development economics                                                      | Research Assistant  | Economics                 | https://www.povertyactionlab.org/careers/research-analyst-global-poverty-research-lab-kellogg-school-management-northwestern              | N/A        | N/A                |

### Web Scraping Section for EJM (Econ Job Market) 🔎

This function is designed to scrape job postings from the Econ Job Market (EJM) page. It performs the following tasks:

- **🌐 Fetching the Page:**  
  It sends an HTTP GET request to the EJM URL using the `requests` library.

- **🥣 Parsing HTML:**  
  The response content is parsed with BeautifulSoup to create a DOM structure for extraction.

- **🔍 Locating Job Panels:**  
  It finds all `<div>` elements with the classes `"panel panel-info"`, each representing a job posting.

- **🏷️ Extracting Job Details:**  
  For each panel, it extracts:
  - **Job Title & Link:** Located within an `<a>` tag with an ID starting with "title-".  
  - **University & Program Type:** Extracted from `<div>` elements with class `"col-md-4"` and `"col-md-2"`, respectively.
  - **Publication Date & Deadline:** Extracted from `<div>` elements with class `"col-md-2"`.
  - **Default Values:** Fields such as **sponsor**, **institution**, and **fields** are set to `"N/A"` since they're not provided.
  
- **🔑 Determining the Main Field:**  
  It combines the program title and university information to deduce the primary research field using the helper function `extract_main_field()`.

- **✅ Building the Result List:**  
  Each job is stored as a dictionary, and all such dictionaries are appended to a list which is then returned.



In [ ]:
import requests
from bs4 import BeautifulSoup
import re

def scrape_ejm():
    """
    Scrapes the Econ Job Market (EJM) page and extracts detailed job information
    from the newer HTML structure. Post-processing steps include:
      - Merging single/multiple salaries into one cell.
      - Parsing sponsor(s) from text referencing "Professors ..."
      - Removing extraneous punctuation in 'fields' (bullet '•', semicolon ';', repeated commas).
      - Replacing 'link' with the final application link (or None if missing).
      - Inheriting 'start_date' if 'Flexible' from a previous non-Flexible record.
    
    Returns a list of dictionaries.
    """
    EJM_URL = "https://econjobmarket.org/market"
    jobs = []
    
    try:
        response = requests.get(EJM_URL)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Each job listing is typically under <div class="panel panel-info">
        panels = soup.find_all("div", class_="panel panel-info")
        for panel in panels:
            job = {}
            job["source"] = "ejm"
            
            # ---------- MAIN ROW (col-md-4, col-md-4, col-md-2, col-md-2) ----------
            main_row = panel.find("div", class_="row")
            if not main_row:
                continue
            
            cols = main_row.find_all("div", recursive=False)
            
            # --- FIRST COLUMN: title, location, start_date, duration ---
            if len(cols) >= 1:
                first_col = cols[0]
                title_a = first_col.find("a", id=lambda x: x and x.startswith("title-"))
                
                if title_a:
                    job["program_title"] = title_a.get_text(strip=True)
                    # We'll store a temporary link here; final link will become 'application_link'
                    job["temp_link"] = title_a.get("href", "").strip()
                else:
                    job["program_title"] = "N/A"
                    job["temp_link"] = ""
                
                col_text = first_col.get_text(separator="\n", strip=True).split("\n")
                # Often line 2 is location
                job["location"] = col_text[1].strip() if len(col_text) >= 2 else None
                
                job["start_date"] = None
                job["duration"] = None
                for line in col_text:
                    lower_line = line.lower()
                    if lower_line.startswith("starts"):
                        clean_line = line.replace("Starts", "").replace(".", "").strip()
                        job["start_date"] = clean_line if clean_line else None
                    elif lower_line.startswith("duration"):
                        clean_line = line.replace("Duration:", "").strip()
                        job["duration"] = clean_line if clean_line else None
            
            # --- SECOND COLUMN: department, university ---
            if len(cols) >= 2:
                second_col = cols[1]
                lines_2 = second_col.get_text(separator="\n", strip=True).split("\n")
                job["department"] = lines_2[0].strip() if len(lines_2) >= 1 else None
                job["university"] = lines_2[1].strip() if len(lines_2) >= 2 else None
            
            # --- THIRD COLUMN: program_type, fields ---
            if len(cols) >= 3:
                third_col = cols[2]
                program_text = third_col.get_text(separator="\n", strip=True).split("\n", 1)
                job["program_type"] = program_text[0].strip() if program_text else None
                
                fields_div = third_col.find("div", id=re.compile(r"cats-\d+"))
                if fields_div:
                    fields_raw = fields_div.get_text(separator=", ", strip=True)
                else:
                    fields_raw = program_text[1].strip() if len(program_text) > 1 else ""
                
                # Clean fields: remove bullet dots, semicolons, repeated commas
                fields_clean = re.sub(r"[•;]", "", fields_raw)
                fields_clean = re.sub(r",\s*,", ",", fields_clean)
                fields_clean = re.sub(r"\s+", " ", fields_clean).strip(" ,")
                job["fields"] = fields_clean if fields_clean else None
            
            # --- FOURTH COLUMN: publication_date, deadline ---
            if len(cols) >= 4:
                fourth_col = cols[3]
                spans = fourth_col.find_all("span")
                
                job["publication_date"] = spans[0].get_text(strip=True) if len(spans) > 0 else None
                job["deadline"] = spans[1].get_text(strip=True) if len(spans) > 1 else None
            else:
                job["program_type"] = job.get("program_type") or None
                job["publication_date"] = None
                job["deadline"] = None
                job["fields"] = job.get("fields") or None
            
            # Placeholders for collapsed info
            job["sponsor"] = None
            job["institution"] = None
            job["main_field"] = None
            job["degree_required"] = None
            job["salary_range"] = None
            job["application_link"] = None
            
            # ---------- COLLAPSE BLOCK (extended info) ----------
            if title_a:
                collapse_id = title_a.get("href", "")
                if collapse_id.startswith("#"):
                    collapse_div_id = collapse_id[1:]
                    collapse_div = panel.find("div", id=collapse_div_id)
                    if collapse_div:
                        # We'll parse the entire collapse text in one go
                        collapse_text = collapse_div.get_text(separator="\n", strip=True)
                        
                        # Parse sponsor(s) from text with "Professors" ...
                        # We'll look for a pattern: "Professors (.*?)." or "Professor (.*?)."
                        # This is a heuristic; adjust to your content.
                        prof_match = re.search(
                            r'(?:[Pp]rofessors?\s+)(.*?)(?:\.|$)', collapse_text
                        )
                        if prof_match:
                            sponsor_str = prof_match.group(1)
                            # Replace ' and ' with comma
                            sponsor_str = sponsor_str.replace(" and ", ", ")
                            # Split by commas
                            sponsor_list = [x.strip() for x in sponsor_str.split(",") if x.strip()]
                            # Re-join with commas
                            job["sponsor"] = ", ".join(sponsor_list)
                        
                        # We'll search within <div> tags with <strong> for structured data
                        additional_divs = collapse_div.find_all("div")
                        for div_item in additional_divs:
                            strong_tag = div_item.find("strong")
                            if strong_tag:
                                label = strong_tag.get_text(strip=True).lower()
                                val = div_item.get_text(separator="\n", strip=True)
                                # remove the strong text from val
                                val = val.replace(strong_tag.get_text(strip=True), "").strip(": \n")
                                
                                if "degree required" in label:
                                    job["degree_required"] = val if val else None
                                elif "job start date" in label:
                                    job["start_date"] = val if val else None
                                elif "job duration" in label:
                                    job["duration"] = val if val else None
                                elif "salary" in label:
                                    # unify multiple lines for salary
                                    raw_lines = val.split("\n")
                                    unified = " ".join(x.strip() for x in raw_lines if x.strip())
                                    job["salary_range"] = unified if unified else None
                        
                        # Try to parse "To Apply" link
                        apply_paragraph = collapse_div.find("p", text=re.compile(r"To\s+Apply", re.IGNORECASE))
                        if apply_paragraph:
                            next_link = apply_paragraph.find_next("a", href=True)
                            if next_link:
                                job["application_link"] = next_link.get("href", None)
                        else:
                            # or search any <a> with 'apply' in text
                            apply_a = collapse_div.find("a", href=True, text=re.compile(r"apply", re.IGNORECASE))
                            if apply_a:
                                job["application_link"] = apply_a.get("href", None)
            
            jobs.append(job)
    
    except Exception as e:
        print("Error during EJM scraping:", e)
    
    # ---------- POST-PROCESSING ----------
    # (1) Replace 'link' with final 'application_link', or None if missing/'https://econjobmarket.org'
    # (2) If 'start_date' == 'Flexible', copy from the nearest preceding non-Flexible record
    for i, job in enumerate(jobs):
        # (1) link substitution
        app_link = job["application_link"]
        if not app_link or app_link.strip() == "" or app_link.strip() == "https://econjobmarket.org":
            job["link"] = None
        else:
            job["link"] = app_link
        
        # (2) if 'start_date' is 'Flexible', inherit from previous
        sd = job.get("start_date")
        if isinstance(sd, str) and sd.lower() == "flexible":
            new_date = None
            # look upward
            for j in range(i-1, -1, -1):
                prev_sd = jobs[j].get("start_date")
                if prev_sd and isinstance(prev_sd, str) and prev_sd.lower() != "flexible":
                    new_date = prev_sd
                    break
            job["start_date"] = new_date  # can be None if never found
    
    # Remove temp columns
    for job in jobs:
        if "temp_link" in job:
            del job["temp_link"]
        if "application_link" in job:
            del job["application_link"]
    
    return jobs


df  = pd.DataFrame(scrape_ejm()).head(10).to_markdown(index=False)

display(Markdown(df))

/var/folders/zd/nzwwg11s06sf2dv5349vbjxh0000gn/T/ipykernel_38845/217879207.py:150: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  apply_paragraph = collapse_div.find("p", text=re.compile(r"To\s+Apply", re.IGNORECASE))
/var/folders/zd/nzwwg11s06sf2dv5349vbjxh0000gn/T/ipykernel_38845/217879207.py:157: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  apply_a = collapse_div.find("a", href=True, text=re.compile(r"apply", re.IGNORECASE))


| source   | program_title                                                                   | location              | start_date   | duration             | department                                                       | university                            | program_type                 | fields                                                                                                                                                                                                                                                                                          | publication_date   | deadline    | sponsor   | institution   | main_field                                                               | degree_required   | salary_range         | link                                                                                                                                                                                                                             |
|:---------|:--------------------------------------------------------------------------------|:----------------------|:-------------|:---------------------|:-----------------------------------------------------------------|:--------------------------------------|:-----------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------------------|:------------|:----------|:--------------|:-------------------------------------------------------------------------|:------------------|:---------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| ejm      | Faculty of Economics at Plaksha University, India                               | Mohali,               |              | Continuing/permanent | Plaksha University                                               |                                       | Various position types       | Development Growth, Econometrics, Environmental Ag. Econ., Experimental Economics, Finance, International Finance/Macro, Macroeconomics Monetary, Behavioral Economics, Any field, Business Economics, Marketing, Operations Research, Statistics, Microeconomic theory, Applied microeconomics | 19 Nov 2024        | 30 Jul 2025 |           |               | Economics, Macroeconomics, Microeconomics, Microeconomic theory, Finance | Doctorate         |                      | https://apply.interfolio.com/39648/positions                                                                                                                                                                                     |
| ejm      | Predoctoral Research Analyst -- Applied Microeconomics                          | Philadelphia,         | 2025-07-01   | 2 years              | Business Economics and Public Policy, Wharton School             | University of Pennsylvania            | Research Assistant (Pre-Doc) | Development Growth, Environmental Ag. Econ., Experimental Economics, Labor Demographic Economics, Public Economics, Behavioral Economics                                                                                                                                                        | 9 Oct 2024         | 1 Jul 2025  |           |               | Economics                                                                | Bachelors         | 55,000 to 60,000 USD | https://wd1.myworkdaysite.com/en-US/recruiting/upenn/careers-at-penn/details/Faculty-Research-Analyst--Business--Economics--and-Public-Policy-Department--Wharton-School_JR00098140-1?jobFamily=ac2a3e0e9a860145e03c7bdc4209c207 |
| ejm      | Research Assistant                                                              | (see ad for location) | 2025-07-01   | 6 months             | Animal Welfare Economics Working Group                           |                                       | Research Assistant           | Any field                                                                                                                                                                                                                                                                                       | 15 Dec 2024        | 15 Feb 2025 |           |               |                                                                          | Bachelors         | 50 USD               | https://docs.google.com/forms/d/e/1FAIpQLSfi18yuyB6O4KiefLlYMqYTvD2HSNtqwpZnW_ZPPUz1cUp4yA/viewform?usp=sf_link                                                                                                                  |
| ejm      | Research Assistant (CAGE)                                                       | Coventry,             | 2025-07-01   | 18 months            | Economics                                                        | University of Warwick                 | Research Assistant           | Any field                                                                                                                                                                                                                                                                                       | 13 Feb 2025        | 17 Mar 2025 |           |               |                                                                          | Masters           |                      |                                                                                                                                                                                                                                  |
| ejm      | PhD scholarships in Economics and Management on Ports as Energy Transition Hubs | Frederiksberg,        | 2025-09-01   | 36 months            | Department of Economics                                          | Copenhagen Business School            | Doctoral student             | Business Economics Management, General                                                                                                                                                                                                                                                          | 22 Jan 2025        | 17 Mar 2025 |           |               | Economics                                                                | Masters           |                      | https://candidate.hr-manager.net/ApplicationInit.aspx?cid=1309&ProjectId=147437&DepartmentId=18993&MediaId=4614&SkipAdvertisement=true                                                                                           |
| ejm      | 1 Research Grant, IGIER Research Center                                         | Milano,               | 2025-05-01   | 36 months            | Economics                                                        | Bocconi University                    | Postdoctoral Scholar         | Labor Demographic Economics, Political Economy, Management, Information Technology                                                                                                                                                                                                              | 16 Dec 2024        | 9 Mar 2025  |           |               | Economics                                                                | Doctorate         |                      |                                                                                                                                                                                                                                  |
| ejm      | Post-Doctoral Associate in the Division of Social Science [Economics]           | Abu Dhabi,            | 2025-05-01   | 2 years              | Economics                                                        | New York University Abu Dhabi         | Research Assistant           | Macroeconomics Monetary                                                                                                                                                                                                                                                                         | 16 Nov 2024        | 1 Dec 2024  |           |               | Economics, Macroeconomics                                                | Doctorate         |                      | https://nyuad.nyu.edu/content/dam/nyuad/about/careers/magazines/NYU-Abu-Dhabi-Compensation-and-Benefits.pdf                                                                                                                      |
| ejm      | Research Assistant                                                              | Boston,               | 2025-05-01   | 1 year               | FutureTech                                                       | Massachusetts Institute of Technology | Research Assistant           | Research Assistant (Pre-Doc) Econometrics                                                                                                                                                                                                                                                       | 15 Jan 2025        | 1 Mar 2025  |           |               |                                                                          | Bachelors         |                      |                                                                                                                                                                                                                                  |
| ejm      | Research Engineer or Postdoctoral position in Health Economics                  | Cergy-Pontoise cedex, | 2025-05-01   | 1 year               | Information Systems, Decision Sciences and Statistics Department | ESSEC Business School                 | Postdoctoral Scholar         | Other academic Research Assistant Econometrics Health Education Welfare                                                                                                                                                                                                                         | 13 Aug 2024        | 15 Sep 2024 |           |               |                                                                          | Masters           |                      | mailto:lamiraud@essec.edu                                                                                                                                                                                                        |
| ejm      | Several postdoctoral researchers in microeconomic theory                        | Bonn,                 | 2025-08-01   | 3 years              | Department of Economics                                          | University of Bonn                    | Postdoctoral Scholar         | Research Assistant Microeconomic theory                                                                                                                                                                                                                                                         | 5 Nov 2024         | 28 Feb 2025 |           |               | Microeconomic theory                                                     | Doctorate         |                      | https://econjobmarket.org/login                                                                                                                                                                                                  |

## Extract Main Field Helper Function 🔑

The `extract_main_field` function analyzes a given text to determine which research fields are mentioned. It searches for multiple keywords in a **case-insensitive** manner. If one or more keywords are found, it returns them as a comma‑separated string. If none are found, it returns `"N/A"`.

### Keywords Included:
- **Economics**
- **Macroeconomics**
- **Microeconomics**
- **Labour**
- **Industrial Organization**
- **Enterpreneurship**
- **Healthcare**
- **Discrimination**
- **Finance**
- **Public Policy**

You can extend this list with additional fields in economics as needed.

In [44]:
def extract_main_field(text):
    """
    Looks for keywords in the provided text.
    Keywords: Economics, Macroeconomics, Microeconomics, Labour, Industrial Organization,
    Enterpreneurship, Healthcare, Discrimination, Finance, Public Policy.
    Returns a comma-separated string of all found keywords or "N/A" if none are found.
    """
    keywords = [
        "Economics", "Macroeconomics", "Microeconomics","Microeconomic theory", "Macroeconomic theory"
        "Labour", "Industrial Organization", "Entrepreneurship",
        "Healthcare", "Discrimination", "Finance", "Public Policy", "Local Economic Policy", "Climate"
    ]
    
    found = []
    for keyword in keywords:
        if keyword.lower() in text.lower():
            found.append(keyword)
    
    if found:
        # Remove duplicates while preserving order and return as a comma-separated string.
        unique_keywords = list(dict.fromkeys(found))
        return ", ".join(unique_keywords)
    else:
        return None

**Function: `extract_program_type(text)`**

- **Purpose:**  
  This function takes a string as input (which might be a job title or description) and determines the program type based on certain keywords.

- **How It Works:**  
  1. **Convert to Lowercase:**  
     The input text is converted to lowercase to ensure case-insensitive matching.
  2. **Keyword Checks:**  
     - If the text contains any variation of "predoctoral" (e.g., "predoctoral", "pre doc", "pre-doc", "predoc"), it returns **"PreDoctoral Program"**.
     - If the text contains any variation of "postdoc" (e.g., "postdoc", "post doc", "post-doc", "postdoctoral", "post doctoral"), it returns **"Post Doc"**.
     - If the text contains "phd" or "ph.d", it returns **"PhD"**.
     - If the text mentions "research assistant" or even "ra" (for example, in abbreviated or extensive form), it returns **"Research Assistant"**.
  3. **Default Category:**  
     If none of the keywords are found, the function defaults to returning **"Research Assistant"**.





In [13]:
def extract_program_type(text):
    """
    Analyzes the provided text (e.g., a job title or description) to determine the program type.
    
    Keywords used:
      - "PreDoctoral Program" if the text includes variations like "predoctoral", "predoc", etc.
      - "Post Doc" if the text includes variations like "postdoc", "post-doctoral", etc.
      - "PhD" if the text includes "phd" or "ph.d".
      - "Research Assistant" (RA) for all other cases.
    
    Returns:
      A string representing the program type.
    """
    # Convert the text to lowercase for case-insensitive matching. 🔍
    text_lower = text.lower()
    
    # Check for PreDoctoral indicators. 🎓
    if any(kw in text_lower for kw in ["predoctoral", "pre doc", "pre-doc", "predoc"]):
        return "PreDoctoral Program"
    
    # Check for Post Doc indicators. 📚
    elif any(kw in text_lower for kw in ["postdoc", "post doc", "post-doc", "postdoctoral", "post doctoral"]):
        return "Post Doc"
    
    # Check for PhD indicators. 🎓
    elif "phd" in text_lower or "ph.d" in text_lower:
        return "PhD"
    
    # Check for Research Assistant indicators. 💼
    elif "research assistant" in text_lower or "ra" in text_lower:
        return "Research Assistant"
    
    # Default category is Research Assistant (RA). 🔄
    else:
        return "Research Assistant"



## CSV & Email Handling Section 📊✉️

This section contains helper functions to manage your job database and send email notifications when new opportunities are detected.

### 1. Reading Existing Jobs from a CSV File 📂

The `read_existing_jobs()` function reads a CSV file that contains saved job listings and returns a **set** of job links that are already recorded.  
- It checks if the file exists.  
- It uses Python's `csv.DictReader` to iterate over rows and collects the "link" field for each job.  

### 2. Appending New Jobs to the CSV File 💾

The `append_jobs_to_csv()` function takes a list of job dictionaries and appends them to the specified CSV file.
- If the CSV file doesn't exist, it creates the file and writes the header.  
- It then appends each job as a new row.  

### 3. Sending Email Notifications ✉️

- **Purpose:**  
  This function sends an email notification whenever new job records are found.  
  - **Single Record:** The subject is set to the university name from that record.
  - **Multiple Records:** The subject lists the unique university names (e.g., "University A, University B").

- **Email Body:**  
  The email body is constructed as an HTML document with a styled table that lists:
  - **Source** (e.g., "predoc", "nber", "ejm")
  - **Program Title**
  - **Clickable Link** (each link is rendered as a clickable hyperlink)
  - **Sponsor**
  - **Institution**
  - **Fields**
  - **Main Field**
  - **Deadline**
  - **University**
  - **Program Type**
  - **Publication Date**

- **How It Works:**  
  1. **Subject Creation:**  
     The function extracts university names from each job record. If there's only one record, it uses that university name; if multiple, it joins all unique names.
  
  2. **HTML Table Construction:**  
     An HTML table is built with one row per job record, ensuring that links are rendered as clickable hyperlinks.
  
  3. **Email Assembly:**  
     The email is composed as a multipart message with both plain text and HTML parts.
  
  4. **Sending the Email:**  
     Using Python's `smtplib`, the function logs in to the SMTP server (defaulting to Gmail) and sends the email.



In [ ]:
def read_existing_jobs(csv_file):
    """
    Reads the CSV file and returns a set of job links that are already recorded.
    """
    existing_links = set()
    if os.path.exists(csv_file):
        with open(csv_file, newline='', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                if "link" in row:
                    existing_links.add(row["link"])
    return existing_links

def append_jobs_to_csv(csv_file, jobs, fieldnames):
    """
    Appends the list of job dictionaries to the CSV file.
    If the file does not exist, it is created and a header is written.
    """
    file_exists = os.path.exists(csv_file)
    with open(csv_file, "a", newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        for job in jobs:
            writer.writerow(job)

def send_email_new_jobs(new_jobs, sender_email, sender_password, receiver_email, smtp_server="smtp.gmail.com", smtp_port=587):
    """
    Sends an email with new job records.
    
    If there is a single record, the email subject is set to the university name from that record.
    If there are multiple records, the subject lists the unique university names (comma-separated).
    
    The email body is an HTML table containing one row per job record.
    Clickable hyperlinks are created for the job links.
    """
    
    # Extract university names from the new jobs (ignoring "N/A")
    universities = [job.get("university", "N/A") for job in new_jobs if job.get("university", "N/A") != "N/A"]
    unique_universities = list(dict.fromkeys(universities))  # preserve order, remove duplicates
    
    # Set email subject based on number of records:
    if len(new_jobs) == 1:
        subject = unique_universities[0] if unique_universities else "New Research Position"
    else:
        subject = ", ".join(unique_universities) if unique_universities else "New Research Positions"
    
    # Construct the HTML table for the email body
    html_body = """
    <html>
      <head>
        <style>
          table, th, td {
            border: 1px solid #ddd;
            border-collapse: collapse;
            padding: 8px;
          }
          th {
            background-color: #f2f2f2;
          }
        </style>
      </head>
      <body>
        <p>New Research Positions Found:</p>
        <table>
          <tr>
            <th>Source</th>
            <th>Program Title</th>
            <th>Link</th>
            <th>Sponsor</th>
            <th>Institution</th>
            <th>Fields</th>
            <th>Main Field</th>
            <th>Deadline</th>
            <th>University</th>
            <th>Program Type</th>
            <th>Publication Date</th>
          </tr>
    """
    for job in new_jobs:
        link = job.get("link", "")
        # Make the link clickable if available
        clickable_link = f'<a href="{link}">{link}</a>' if link else "N/A"
        html_body += f"""
          <tr>
            <td>{job.get("source", "N/A")}</td>
            <td>{job.get("program_title", "N/A")}</td>
            <td>{clickable_link}</td>
            <td>{job.get("sponsor", "N/A")}</td>
            <td>{job.get("institution", "N/A")}</td>
            <td>{job.get("fields", "N/A")}</td>
            <td>{job.get("main_field", "N/A")}</td>
            <td>{job.get("deadline", "N/A")}</td>
            <td>{job.get("university", "N/A")}</td>
            <td>{job.get("program_type", "N/A")}</td>
            <td>{job.get("publication_date", "N/A")}</td>
          </tr>
        """
    html_body += """
        </table>
      </body>
    </html>
    """
    
    # Create a multipart email message (plain text and HTML)
    msg = MIMEMultipart("alternative")
    msg["Subject"] = subject
    msg["From"] = sender_email
    msg["To"] = receiver_email
    
    # Plain text version as fallback
    text_body = "New Research Positions Found. Please view this email in an HTML-compatible client."
    
    part1 = MIMEText(text_body, "plain")
    part2 = MIMEText(html_body, "html")
    
    msg.attach(part1)
    msg.attach(part2)
    
    # Send the email via SMTP
    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()  # Secure the connection
            server.login(sender_email, sender_password)
            server.send_message(msg)
        print("Email sent successfully!")
    except Exception as e:
        print("Failed to send email:", e)

## Main Function: Scrape, Update, and Notify 🚀📊✉️

This **main()** function orchestrates the complete workflow of the project. It:

- **Scrapes Job Data:**  
  Calls the scraping functions for all three sources (Predoc, NBER, EJM) to collect job postings.

- **Filters New Jobs:**  
  Reads an existing CSV file (acting as a simple database) to get a set of already recorded job links. Then, it filters out jobs that are already present.

- **Sends Notifications:**  
  For each new job found, the function sends an email notification with the job details.

- **Updates the CSV Database:**  
  Finally, it appends the new job entries to the CSV file for future reference.

> **Note:**  
> Ensure that your SMTP credentials (i.e. `SENDER_EMAIL` and `SENDER_PASSWORD`) are set up and that the scraping functions (`scrape_predoc()`, `scrape_nber()`, and `scrape_ejm()`) along with CSV and email helper functions are defined before running `main()`.

In [ ]:
def main():
    # Scrape jobs from all three sources. 🌐
    jobs = []
    jobs.extend(scrape_predoc())
    jobs.extend(scrape_nber())
    jobs.extend(scrape_ejm())

    # Check if any jobs were scraped. 🚨
    if not jobs:
        print("No jobs were scraped.")
        return

    # Read existing jobs from CSV (using 'link' as a unique identifier). 📂
    existing_links = read_existing_jobs(CSV_FILE)

    # Filter out jobs that are already recorded. 🔍
    new_jobs = [job for job in jobs if job.get("link") not in existing_links]
    print(f"Found {len(new_jobs)} new job(s).")

    # Define CSV columns (fieldnames) for consistent data structure. 📋
    fieldnames = [
        "source", "program_title", "link", "sponsor",
        "institution", "fields", "main_field", "deadline",
        "university", "program_type", "publication_date"
    ]

    # If new jobs are found, process them. ✉️💾
    if new_jobs:
        # Retrieve SMTP credentials from environment variables.
        sender_email    = os.getenv('SENDER_EMAIL')
        sender_password = os.getenv('SENDER_PASSWORD')
        receiver_email  = os.getenv('SENDER_EMAIL')
        
        # Convert the new jobs to a Pandas DataFrame for easy visualization. 📈
        df = pd.DataFrame(new_jobs)

        md_table = df.to_markdown(index=False)
        # Uncomment these lines if you want to send an email and update the CSV.
        # send_email_new_jobs(new_jobs, sender_email, sender_password, receiver_email)
        # append_jobs_to_csv(CSV_FILE, new_jobs, fieldnames)
        
        # Print the DataFrame to visualize the new jobs.
        
    else:
        # If no new jobs were found, print a message and display existing links.
        print("No new jobs found.")
        df = pd.read_csv(CSV_FILE)
        md_table = df.to_markdown(index=False)
    # Display the table in the notebook
    display(Markdown(md_table))


In [ ]:
if __name__ == "__main__":
    main()


Found 0 new job(s).
No new jobs found.


| source   | program_title                                                                                             | link                        | sponsor                                                                                                                                                                                                                                                                                                                                                                                                                                                                | institution                                                                                                                                                                                | fields                                                                                                                                                                                                                                                               | main_field                                                         | deadline                                                                                                                                                                                                                              | university                                                                             | program_type                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | publication_date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|:---------|:----------------------------------------------------------------------------------------------------------|:----------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------------------------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:---------------------------------------------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| predoc   | Research Associates                                                                                       | https://bit.ly/3Xhfcvq      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | Education, Finance, Labor, Macro, Public Policy, and Urban                                                                                                                                                                                                           | Finance, Public Policy                                             | Rolling In addition to a multi-institutional job pool, this website also provides general information about opportunities within the Federal Reserve System as well as up-to-date information re specific openings.                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Positions                                                                                    | https://bit.ly/3CM4IxV      | Abhishek Nagaraj (UC Berkeley) and Matteo Tranchero (Penn)                                                                                                                                                                                                                                                                                                                                                                                                             | UC Berkeley’s Haas School of Business; Wharton School of the University of Pennsylvania                                                                                                    | Applied economics, innovation, entrepreneurship, data science and tech innovation                                                                                                                                                                                    | Economics                                                          | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Technical Associate                                                                          | https://bit.ly/4hBFgtU      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Technical Associate                                                                          | https://bit.ly/4er9iOY      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Associate in Economics                                                              | https://bit.ly/4jQlfBs      | Matthew Pecenco                                                                                                                                                                                                                                                                                                                                                                                                                                                        | Brown University                                                                                                                                                                           | Labor, Crime, Housing                                                                                                                                                                                                                                                | Economics                                                          | Applications will be accepted and reviewed on a rolling basis.                                                                                                                                                                        | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Full-Time Research Assistant                                                                              | https://bit.ly/4hwY3a0      | Matthew Baron                                                                                                                                                                                                                                                                                                                                                                                                                                                          | National Bureau of Economic Research                                                                                                                                                       | Banking, Financial Crises, Financial History                                                                                                                                                                                                                         | nan                                                                | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Fellowship                                                                                   | https://bit.ly/3WXU7GY      | Hans-Joachim Voth                                                                                                                                                                                                                                                                                                                                                                                                                                                      | University of Zurich                                                                                                                                                                       | Economic History, Political Economy, Cultural Economics                                                                                                                                                                                                              | Economics                                                          | Applications will be reviewed immediately and are welcome until all positions are filled.                                                                                                                                             | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Professional in Accounting                                                                       | https://bit.ly/4hyJouK      | Professor Philip Berger                                                                                                                                                                                                                                                                                                                                                                                                                                                | The University of Chicago Booth School of Business                                                                                                                                         | Accounting, corporate finance, and labor economics.                                                                                                                                                                                                                  | Economics, Finance                                                 | Applications are reviewed on a rolling basis; the initial full review will be March 15, 2025.                                                                                                                                         | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Professional in Marketing                                                                        | https://bit.ly/3Eupeng      | Professor Andreas Kraft                                                                                                                                                                                                                                                                                                                                                                                                                                                | University of Chicago Booth School of Business                                                                                                                                             | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Professional in Behavioral Science                                                               | https://bit.ly/3QkttnL      | Professor Alexander Todorov                                                                                                                                                                                                                                                                                                                                                                                                                                            | University of Chicago Booth School of Business                                                                                                                                             | Behavioral Science                                                                                                                                                                                                                                                   | nan                                                                | Applications are reviewed on a rolling basis; the initial full review will be March 1, 2025.                                                                                                                                          | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Full-Time Research Specialist                                                                             | https://bit.ly/41c0HfB      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | Crockett Lab, Princeton's Department of Psychology and University Center for Human Values                                                                                                  | Psychology, cognitive science, computational social science, data science, and related fields. Project Scope: The Research Specialist will contribute to research projects investigating transformative experience, identity change, and empathy in the digital age. | nan                                                                | Applications will go under review starting February 14th, rolling afterward.                                                                                                                                                          | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Fellowship in Applied Economics                                                              | https://bit.ly/4jMh0XH      | Profs. Nate Baum-Snow, Victor Couture, and Stephan Heblich                                                                                                                                                                                                                                                                                                                                                                                                             | Department of Economics and Rotman School of Management, University of Toronto                                                                                                             | nan                                                                                                                                                                                                                                                                  | Economics                                                          | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Fellow                                                                              | https://bit.ly/40MdGDf      | Dr. John M. Griffin (McCombs School of Business)                                                                                                                                                                                                                                                                                                                                                                                                                       | Integra FEC                                                                                                                                                                                | Finance, CMBS, Economics, Cryptocurrency, & Fraud Analytics                                                                                                                                                                                                          | Economics, Finance                                                 | 04/04/2025                                                                                                                                                                                                                            | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Associate                                                                           | https://bit.ly/4jHBP6J      | Paul A. Gompers                                                                                                                                                                                                                                                                                                                                                                                                                                                        | Harvard Business School                                                                                                                                                                    | Entrepreneurship and Finance                                                                                                                                                                                                                                         | Finance                                                            | Applications will be accepted and reviewed on a rolling basis.                                                                                                                                                                        | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Fellow                                                                                       | https://bit.ly/3CtAoIk      | Reed Walker (Business & Public Policy)                                                                                                                                                                                                                                                                                                                                                                                                                                 | The Center for Effective Global Action (CEGA) and the Opportunity Lab at UC Berkeley                                                                                                       | Energy and Environmental Economics; Labor Economics; Public Economics First Review: February 18, 2025                                                                                                                                                                | Economics                                                          | March 24, 2025                                                                                                                                                                                                                        | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Professional                                                                                     | https://bit.ly/40OKNWZ      | Kawin Ethayarajh                                                                                                                                                                                                                                                                                                                                                                                                                                                       | The University of Chicago Booth School of Business                                                                                                                                         | Microeconomics, Data Science and Tech Innovation, AI and Machine Learning                                                                                                                                                                                            | Economics, Microeconomics                                          | March 31, then rolling                                                                                                                                                                                                                | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Specialist                                                                          | https://bit.ly/42LNHhR      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Researcher                                                                                   | https://bit.ly/4jMRAZW      | Prof. Nina Roussille                                                                                                                                                                                                                                                                                                                                                                                                                                                   | Massachusetts Institute of Technology                                                                                                                                                      | Research in labor economics on the determinants of imperfect competition, gender and racial inequality, as well as the role of information frictions in generating income inequality.                                                                                | Economics                                                          | Rolling Start Date: June 1st, 2025. Job ID: #24818                                                                                                                                                                                    | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Professional                                                                                     | https://bit.ly/40POY4Y      | Christopher Stewart                                                                                                                                                                                                                                                                                                                                                                                                                                                    | The University of Chicago Booth School of Business                                                                                                                                         | Accounting                                                                                                                                                                                                                                                           | nan                                                                | Rolling review, initial full review March 1st                                                                                                                                                                                         | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Professional                                                                                     | https://bit.ly/40M5pz6      | Quentin Vandeweyer and Anthony Lee Zhang                                                                                                                                                                                                                                                                                                                                                                                                                               | The University of Chicago Booth School of Business                                                                                                                                         | Microeconomics, Finance, Economic Theory, Structural Estimation                                                                                                                                                                                                      | Economics, Microeconomics, Finance                                 | February 28, then rolling                                                                                                                                                                                                             | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Fellowship                                                                                   | https://bit.ly/4gLIhXH      | Professor Amanda Pallais                                                                                                                                                                                                                                                                                                                                                                                                                                               | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | Applications will be accepted and reviewed on a rolling basis.                                                                                                                                                                        | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Fellowship                                                                                   | https://bit.ly/3ErL6Q3      | Economics professor Stefanie Stantcheva                                                                                                                                                                                                                                                                                                                                                                                                                                | Social Economics Lab , Harvard University                                                                                                                                                  | Public economics, with a particular focus on taxation, innovation, and people’s attitudes towards public policies.                                                                                                                                                   | Economics                                                          | Applications will be evaluated on a rolling basis until the position is filled. Visa Sponsorship: Available                                                                                                                           | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Fellow                                                                                       | https://bit.ly/4hlbauQ      | Professors Lisa B. Kahn and Ronni Pavan                                                                                                                                                                                                                                                                                                                                                                                                                                | University of Rochester; Census Research Data Center                                                                                                                                       | Labor and urban economics.                                                                                                                                                                                                                                           | Economics                                                          | Review of applications will begin on March 1 2025, with rolling reviews until the position is filled.                                                                                                                                 | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Professional-Climate Impact Lab                                                                  | https://bit.ly/4jIvasG      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Fellow                                                                                           | https://bit.ly/4hcRgC2      | Research Fellow                                                                                                                                                                                                                                                                                                                                                                                                                                                        | Centre for the Governance of AI                                                                                                                                                            | AI regulation, international governance, technical AI governance, economics of AI, AI risk assessment and forecasting                                                                                                                                                | Economics                                                          | Sunday 16 February                                                                                                                                                                                                                    | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Scholar                                                                                          | https://bit.ly/4hcRgC2      | Research Scholar                                                                                                                                                                                                                                                                                                                                                                                                                                                       | Centre for the Governance of AI                                                                                                                                                            | AI regulation, international governance, technical AI governance, economics of AI, AI risk assessment and forecasting.                                                                                                                                               | Economics                                                          | Sunday 16 February                                                                                                                                                                                                                    | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Analyst                                                                                          | https://bit.ly/40z56aO      | Dean Karlan; Christopher Udry                                                                                                                                                                                                                                                                                                                                                                                                                                          | Global Poverty Research Lab, Northwestern University                                                                                                                                       | Development Economics                                                                                                                                                                                                                                                | Economics                                                          | 2/14/25                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Associate in Economics                                                              | https://bit.ly/3WHhxA6      | Professor David Dorn                                                                                                                                                                                                                                                                                                                                                                                                                                                   | The Department of Economics of the University of Zurich                                                                                                                                    | Labor Economics                                                                                                                                                                                                                                                      | Economics                                                          | 28 February 2025                                                                                                                                                                                                                      | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-doctoral Research Assistant                                                                           | https://bit.ly/3CmoZK9      | Assistant Professor Amanda Dahlstrand                                                                                                                                                                                                                                                                                                                                                                                                                                  | The University of Zurich                                                                                                                                                                   | The research assistant will assist in research projects that study health care provision and the role of digitization.                                                                                                                                               | nan                                                                | February 14, 2025. Location: Stockholm, Sweden                                                                                                                                                                                        | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Professional                                                                                     | https://bit.ly/42mLf1b      | Professor Christopher Blattman                                                                                                                                                                                                                                                                                                                                                                                                                                         | BFI’s Development Economics Initiative, The Becker Friedman Institute for Economics at the University of Chicago                                                                           | Violence prevention, the economics of organized crime, and other topics on the intersection of development economics and political economy.                                                                                                                          | Economics                                                          | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Professional                                                                                     | https://bit.ly/4htFJhk      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Kabir Banerjee Pre-Doctoral Fellowship                                                                    | https://bit.ly/4juabdh      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | The Weiss Fund for Research in Development Economics                                                                                                                                       | The Weiss Fund for Research in Development Economics supports innovative research projects that aim to improve the lives of people living in developing countries.                                                                                                   | Economics                                                          | The application deadline is January 31st, 2025 EOD Central Time.                                                                                                                                                                      | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Associate                                                                           | https://bit.ly/4atRIJt      | Dan Garrett and Marius Guenzel                                                                                                                                                                                                                                                                                                                                                                                                                                         | The Wharton School, University of Pennsylvania                                                                                                                                             | Finance                                                                                                                                                                                                                                                              | Finance                                                            | Rolling with First Review Date in early February                                                                                                                                                                                      | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Assistant III                                                                                    | https://bit.ly/40ImZVS      | Nicole Maestas                                                                                                                                                                                                                                                                                                                                                                                                                                                         | Harvard Medical School                                                                                                                                                                     | Health, Labor, Economics                                                                                                                                                                                                                                             | Economics                                                          | Applications reviewed on a rolling basis.                                                                                                                                                                                             | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Fellow                                                                                       | https://bit.ly/42mCkN2      | Bruce D. Meyer                                                                                                                                                                                                                                                                                                                                                                                                                                                         | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | Applications will be reviewed on a rolling basis                                                                                                                                                                                      | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Assistant-Exploring Economics of Technological Progress, Innovation, and Artificial Intelligence | https://bit.ly/40rs6Z7      | Danial Lashkari, Neil Thompson                                                                                                                                                                                                                                                                                                                                                                                                                                         | MIT FutureTech at MIT Sloan                                                                                                                                                                | Innovation and Technical Growth, Macroeconomics, Econometrics, Data Science                                                                                                                                                                                          | Economics, Macroeconomics                                          | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Associate                                                                           | https://bit.ly/40B8Iuc      | Amy W. Huber, Karen K. Lewis, and Yao Zeng                                                                                                                                                                                                                                                                                                                                                                                                                             | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | Rolling with First Review Date in early February                                                                                                                                                                                      | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Full-Time Research Assistant                                                                              | https://bit.ly/42oC2Wd      | Yueran Ma                                                                                                                                                                                                                                                                                                                                                                                                                                                              | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Predoctoral Fellow                                                                                        | https://bit.ly/48DPIgc      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | Opportunity Insights                                                                                                                                                                       | Economics, Public Economics, Big Data, Statistics, Economics Mobility, Labor Markets, Econometrics, Education, Social Capital                                                                                                                                        | Economics                                                          | February 11, 2025                                                                                                                                                                                                                     | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Junior Research Scientist                                                                                 | https://bit.ly/3C4DsdD      | Johannes Stroebel and Theresa Kuchler                                                                                                                                                                                                                                                                                                                                                                                                                                  | Stern School of Business, New York University                                                                                                                                              | Economics, Finance                                                                                                                                                                                                                                                   | Economics, Finance                                                 | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Lab Manager/Research Assistant                                                                            | https://bit.ly/40fXIkD      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | Behavioral Science, Mental Health, Social Neuroscience, Computational Neuroscience.                                                                                                                                                                                  | nan                                                                | Ongoing.                                                                                                                                                                                                                              | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Data Analyst (Pre-Doctoral)                                                                      | https://bit.ly/3WfDMwM      | Ulrike Malmendier                                                                                                                                                                                                                                                                                                                                                                                                                                                      | UC Berkeley, O'Donnell Center for Behavioral Economics (Haas School of Business & Department of Economics)                                                                                 | Applied Microeconomics, Behavioral Economics, Behavioral Finance, Macro-Finance, Labor, Health                                                                                                                                                                       | Economics, Microeconomics, Finance                                 | Rolling. Early applications are encouraged. Job ID: #75137                                                                                                                                                                            | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Assistant                                                                           | https://bit.ly/4gSNIVh      | Davide Cantoni and Claudia Steinwender                                                                                                                                                                                                                                                                                                                                                                                                                                 | Ludwig-Maximilians-University Munich                                                                                                                                                       | Economics, Economic History, Applied Economics                                                                                                                                                                                                                       | Economics                                                          | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Associates                                                                          | https://bit.ly/4hhX1xR      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | Harvard Business School                                                                                                                                                                    | Strategy, Finance, Entrepreneurial Management, Technology and Operations Management, Organizations and Markets, Organizational Behavior, Marketing, or Accounting                                                                                                    | Finance                                                            | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Associate-Junior Professional Researcher Program                                                 | https://bit.ly/40vuEXH      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | Feb 02, 2025 at 11:59 PM Eastern Time                                                                                                                                                                                                 | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Fellow in Financial Economics                                                       | https://bit.ly/4abwboO      | Gordon Phillips and Franz Hinzen                                                                                                                                                                                                                                                                                                                                                                                                                                       | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | Economics                                                          | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Assistant in Economics                                                                           | https://bit.ly/4j72465      | Arash Nekoei and Josef Sigurdsson                                                                                                                                                                                                                                                                                                                                                                                                                                      | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | Economics                                                          | 2025/1/17                                                                                                                                                                                                                             | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Assistants                                                                                       | https://bit.ly/4g49CnD      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | Federal Reserve Bank of Philadelphia                                                                                                                                                       | Economics, monetary policy, econometrics and statistics                                                                                                                                                                                                              | Economics                                                          | Apply by May 15, 2024.                                                                                                                                                                                                                | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Assistant                                                                                        | https://bit.ly/3DKdbBP      | Institute Director and associated economists                                                                                                                                                                                                                                                                                                                                                                                                                           | The Opportunity and Inclusive Growth Institute, Federal Reserve Bank of Minneapolis                                                                                                        | Economic opportunity and inclusive growth, econometrics and statistics                                                                                                                                                                                               | nan                                                                | Late winter or early spring                                                                                                                                                                                                           | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Associate                                                                                        | https://bit.ly/4iB88Uh      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | The Indian School of Business (ISB) Sponsoring Researcher(s): Professor Sabareesh Ramachandran                                                                                             | Economics and Public Policy Project Scope: Professor Sabareesh's research focuses on identifying ways governments should best allocate resources to improve welfare outcomes like education, health, and social protection.                                          | Economics, Public Policy                                           | Rolling. Early applications are encouraged.                                                                                                                                                                                           | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Empirical Research Fellow                                                                                 | https://bit.ly/3CjUYdD      | Dashun Wang                                                                                                                                                                                                                                                                                                                                                                                                                                                            | Kellogg School of Management, Northwestern University                                                                                                                                      | Computational social science, network analysis, metascience                                                                                                                                                                                                          | nan                                                                | Apply by February 7, 2025                                                                                                                                                                                                             | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Specialist                                                                          | https://bit.ly/3We19ai      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | Wharton School of Business, The University of Pennsylvania                                                                                                                                 | AI; Education; Machine Learning Project Scope: The Pre-Doctoral Research Specialist will conduct research on two main projects - understanding impact of AI on human interactions and impact of smartphone bans on educational outcomes.                             | nan                                                                | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Coordinator                                                                                      | https://bit.ly/3DOJ2Bq      | LGBTQI+ and Neurodiversity Research Program                                                                                                                                                                                                                                                                                                                                                                                                                            | Children's National Hospital                                                                                                                                                               | LGBTQIA+ Neurodiversity                                                                                                                                                                                                                                              | nan                                                                | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | CSAP Predoc: Domestic and Foreign Politics of China                                                       | https://bit.ly/3DXyAaw      | Daniel Mattingly                                                                                                                                                                                                                                                                                                                                                                                                                                                       | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Predoctoral Research Associate                                                                            | https://bit.ly/3C1jQae      | Finance unit                                                                                                                                                                                                                                                                                                                                                                                                                                                           | Harvard Business School                                                                                                                                                                    | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Predoctoral Research Associate                                                                            | https://bit.ly/4h2klj8      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | Harvard Business School (HBS)                                                                                                                                                              | Design and management of negotiations, organizations, and markets.                                                                                                                                                                                                   | nan                                                                | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Predoctoral Research Associate                                                                            | https://bit.ly/4fTwIwN      | Aliya Korganbekova and Jungkoo Kang                                                                                                                                                                                                                                                                                                                                                                                                                                    | Harvard Business School                                                                                                                                                                    | Alternative/big data, corporate social responsibility, and corporate finance                                                                                                                                                                                         | Finance                                                            | Applications will be considered on a rolling basis.                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Predoctoral Research Associate                                                                            | https://bit.ly/4j7fVsZ      | Bill Kerr                                                                                                                                                                                                                                                                                                                                                                                                                                                              | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | The recruiting process is rolling. Applications will be reviewed as they are received, now through spring 2025.                                                                                                                       | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Predoctoral Research Associate                                                                            | https://bit.ly/3PoCBau      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | Accounting and Management Unit,, Harvard Business School                                                                                                                                   | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Full-Time Research Assistant in Health Economics and Econometrics                                         | https://bit.ly/42PfuMk      | Professors Amanda Kowalski (Michigan) and David Chan (Stanford)                                                                                                                                                                                                                                                                                                                                                                                                        | University of Michigan in Ann Arbor Project Title: “Equality vs. Equity by Social Vulnerability, Race, and Ethnicity: Evidence from Randomized and Targeted COVID-19 Vaccine Invitations.” | Health Economics; Statistics and Econometrics                                                                                                                                                                                                                        | Economics                                                          | Applications will be considered on a rolling basis.                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Fellow/Researcher                                                                   | https://bit.ly/3ZM0CNC      | Andrew Scott                                                                                                                                                                                                                                                                                                                                                                                                                                                           | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Professional                                                                                     | https://bit.ly/3ZJfqMU      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Empirical Research Fellow                                                                                 | https://bit.ly/40B7pMh      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | Kellogg School of Management, Northwestern University                                                                                                                                      | Quantitative Marketing, Machine Learning, Search                                                                                                                                                                                                                     | nan                                                                | 12/31/2024                                                                                                                                                                                                                            | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Junior Research Scientist                                                                                 | https://bit.ly/4gxyqF9      | Robert Richmond, Toomas Laarits                                                                                                                                                                                                                                                                                                                                                                                                                                        | NYU Stern School of Business                                                                                                                                                               | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Predoctoral Research Fellow Opportunity                                                                   | https://bit.ly/4dgHTQ0      | Professor Will Dobbie (Harvard Kennedy School) and Professor Crystal Yang (Harvard Law School)                                                                                                                                                                                                                                                                                                                                                                         | Malcolm Wiener Center for Social Policy, Harvard Kennedy School                                                                                                                            | Race, discrimination, criminal justice system, and algorithms                                                                                                                                                                                                        | Discrimination                                                     | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Empirical Research Fellow                                                                                 | https://bit.ly/40B7pMh      | Tom Hagenberg, Jung Min Kim, Fabian Nagel, Georg Rickmann                                                                                                                                                                                                                                                                                                                                                                                                              | Kellogg School of Management, Northwestern University                                                                                                                                      | Accounting, information disclosures, credit markets                                                                                                                                                                                                                  | nan                                                                | December 31                                                                                                                                                                                                                           | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Full-Time Research Assistant                                                                              | https://bit.ly/3D1MPLc      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Empirical Research Fellow                                                                                 | https://bit.ly/40B7pMh      | Maria Ibañez                                                                                                                                                                                                                                                                                                                                                                                                                                                           | Kellogg School of Management, Northwestern University                                                                                                                                      | Operations management, labor, health operations                                                                                                                                                                                                                      | nan                                                                | December 31                                                                                                                                                                                                                           | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Fellow/Research Technician                                                                   | https://bit.ly/3ZIZXOl      | Sam Norris, Katherine Wagner, Réka Juhász, Gorkem Bostanci, and others                                                                                                                                                                                                                                                                                                                                                                                                 | Vancouver School of Economics, University of British Columbia                                                                                                                              | nan                                                                                                                                                                                                                                                                  | Economics                                                          | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Associate                                                                           | https://bit.ly/41qvHcl      | Professor Ebehi Iyoha                                                                                                                                                                                                                                                                                                                                                                                                                                                  | Harvard Business School (HBS)                                                                                                                                                              | Research projects are related to international trade and draw heavily on applied econometrics and microeconomic theory.                                                                                                                                              | nan                                                                | The recruiting process is rolling. Applications will be reviewed as they are received, now through spring 2025.                                                                                                                       | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Assistants                                                                                       | https://bit.ly/3ZjsF70      | Dr Chris Colvin and Professor John Turner                                                                                                                                                                                                                                                                                                                                                                                                                              | Centre for Economics, Policy and History at Queen’s University Belfast                                                                                                                     | Economic History                                                                                                                                                                                                                                                     | Economics                                                          | 6 January 2025                                                                                                                                                                                                                        | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Associate                                                                           | https://bit.ly/3ZB5pTi      | Ben Castleman, Tim Layton, Andrew Simon, Sebastian Tello-Trillo, Derek Wu                                                                                                                                                                                                                                                                                                                                                                                              | University of Virginia, Batten School of Leadership and Public Policy                                                                                                                      | Public Economics, Economics of Education, Health Economics, Social Insurance                                                                                                                                                                                         | Economics, Public Policy                                           | January 6, 2025                                                                                                                                                                                                                       | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Associate                                                                           | https://bit.ly/3ZDMU0z      | Professor Ray Kluender                                                                                                                                                                                                                                                                                                                                                                                                                                                 | Harvard Business School                                                                                                                                                                    | Household finance, public economics, and entrepreneurship                                                                                                                                                                                                            | Economics, Finance                                                 | Rolling Start Date: July 1st, 2025.                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Fellowship in Applied Microeconomics: Education, Governance, and Migration                   | https://bit.ly/49oY4JB      | Profs. Ceren Baysan and Román Andrés Zárate                                                                                                                                                                                                                                                                                                                                                                                                                            | Forward Society Lab, Department of Economics, University of Toronto                                                                                                                        | Education, Political Economy, Migration                                                                                                                                                                                                                              | Economics, Microeconomics                                          | Rolling. Apply ASAP for early review.                                                                                                                                                                                                 | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Researcher-Technical Associate                                                               | https://bit.ly/3B7yo7P      | Namrata Kala, Alexey Makarin, Benjamin Vatter, and Catherine Wolfram                                                                                                                                                                                                                                                                                                                                                                                                   | MIT Sloan School of Management                                                                                                                                                             | Applied Microeconomics                                                                                                                                                                                                                                               | Economics, Microeconomics                                          | Applications will be considered on a rolling basis.                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Empirical Research Fellow                                                                                 | https://bit.ly/40B7pMh      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | 12/31/2024                                                                                                                                                                                                                            | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Fellow                                                                                       | https://bit.ly/3YZUgtO      | Jonathan Colmer and Jay Shimshack                                                                                                                                                                                                                                                                                                                                                                                                                                      | University of Virginia/The Environmental Inequality Lab                                                                                                                                    | Environmental Economics                                                                                                                                                                                                                                              | Economics                                                          | 12/1/2024                                                                                                                                                                                                                             | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Fellowship                                                                                   | https://bit.ly/3ZjUaOU      | Professor Gabriel Kreindler                                                                                                                                                                                                                                                                                                                                                                                                                                            | Harvard University                                                                                                                                                                         | Urban, spatial and development economics                                                                                                                                                                                                                             | Economics                                                          | Applications will be accepted and reviewed on a rolling basis.                                                                                                                                                                        | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Professional                                                                                     | https://bit.ly/3YY44UV      | Matthew Notowidigdo and Heather Sarsons                                                                                                                                                                                                                                                                                                                                                                                                                                | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | First review on December 13, 2024, then rolling                                                                                                                                                                                       | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Fellowship in Behavioral-Economic Theory                                                     | https://bit.ly/4fCW3vS      | Professor Matthew Rabin                                                                                                                                                                                                                                                                                                                                                                                                                                                | Harvard University                                                                                                                                                                         | Psychology; Economics                                                                                                                                                                                                                                                | Economics                                                          | Applications will be accepted and reviewed on a rolling basis. The position is ideal for candidates interested in pursuing PhD studies in Economics, particularly those with an interest in psychologically-grounded economic theory. | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Empirical Research Fellow                                                                                 | https://bit.ly/40B7pMh      | Klaus Weber                                                                                                                                                                                                                                                                                                                                                                                                                                                            | Kellogg School of Management, Northwestern University                                                                                                                                      | Computational sociology, linguistics, organizational behavior, sustainability                                                                                                                                                                                        | nan                                                                | December 31                                                                                                                                                                                                                           | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doc Research Analyst                                                                                  | https://bit.ly/4df8Kf4      | Joseph Gyourko, Gilles Duranton, Todd Sinai, Fernando Ferreira, Maisy Wong, Ben Keys, Jessie Handbury, Nina Harari, Sophie Calder-Wang                                                                                                                                                                                                                                                                                                                                 | Wharton Real Estate Center                                                                                                                                                                 | Applied Microeconomics, real estate, urban economics, household finance, public finance, industrial organization, development                                                                                                                                        | Economics, Microeconomics, Industrial Organization, Finance        | Rolling Reference #: JR00094334                                                                                                                                                                                                       | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doc Research Analyst                                                                                  | https://bit.ly/3XkpGu1      | Joseph Gyourko, Maisy Wong, Gilles Duranton, Jessie Handbury                                                                                                                                                                                                                                                                                                                                                                                                           | Zell/Lurie Real Estate Center at the Wharton School of Business, University of Pennslyvania                                                                                                | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Junior Research Scientist                                                                                 | https://bit.ly/3CEwfB2      | Sinja Leonelli, Irina Luneva, Gurpal Sran, and Aleksander Aleszczyk                                                                                                                                                                                                                                                                                                                                                                                                    | NYU Stern School of Business                                                                                                                                                               | Accounting                                                                                                                                                                                                                                                           | nan                                                                | Applications are invited on a rolling basis until December 31, 2024.                                                                                                                                                                  | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Analyst Program (Potential PhD Funding)                                                          | https://bit.ly/4fW9yql      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | International Monetary Fund and Japan-IMF scholarship                                                                                                                                      | Macroeconomics, International Economics, Data Science and Tech Innovation, Finance, Labor, Political Economy, Public Economics, Public Policy, Statistics and Econometrics, Urban                                                                                    | Economics, Macroeconomics, Finance, Public Policy                  | Rolling PhD Funding: Ph.D. funding is potentially available for Japanese nationals only                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Fellow                                                                              | https://bit.ly/4eE1auv      | Professors Stefanie Deluca and Nick Papageorge                                                                                                                                                                                                                                                                                                                                                                                                                         | Poverty and Inequality Research Lab at Johns Hopkins University                                                                                                                            | Sociology and Economics                                                                                                                                                                                                                                              | Economics                                                          | December 2, 2024 Position Overview                                                                                                                                                                                                    | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Data Analyst Associate                                                                           | https://bit.ly/3YUk8XI      | Charles Manski                                                                                                                                                                                                                                                                                                                                                                                                                                                         | Northwestern University, Department of Economics                                                                                                                                           | Economics, Health                                                                                                                                                                                                                                                    | Economics                                                          | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Associate                                                                                        | https://bit.ly/3YHu7iY      | Gideon Bornstein, Sasha Indarte, Sergio Salgado                                                                                                                                                                                                                                                                                                                                                                                                                        | Wharton School/University of Pennsylvania                                                                                                                                                  | Macroeconomics, household finance and inequality                                                                                                                                                                                                                     | Economics, Macroeconomics, Finance                                 | Applications received by November 26, 2024 will be given priority.                                                                                                                                                                    | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Fellow                                                                              | https://bit.ly/3YQlUcl      | Elaine Hill                                                                                                                                                                                                                                                                                                                                                                                                                                                            | University of Rochester Health and Environmental Economics Lab                                                                                                                             | nan                                                                                                                                                                                                                                                                  | Economics                                                          | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Assistant                                                                           | https://bit.ly/3At5Qp4      | Research Department                                                                                                                                                                                                                                                                                                                                                                                                                                                    | International Monetary Fund                                                                                                                                                                | Macroeconomics, Finance, Public Economics                                                                                                                                                                                                                            | Economics, Macroeconomics, Finance                                 | 20 November 2024                                                                                                                                                                                                                      | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Fellow                                                                              | https://bit.ly/3CjDzlr      | Azeem Shaikh and Max Tabord-Meehan                                                                                                                                                                                                                                                                                                                                                                                                                                     | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Professional                                                                                 | https://bit.ly/4fGSBQC      | Professors Elisa Jácome and Silvia Vannutelli                                                                                                                                                                                                                                                                                                                                                                                                                          | Department of Economics, Northwestern University                                                                                                                                           | Labor economics, public economics, applied microeconomics, economic history, mobility and inequality, socio-political analyses                                                                                                                                       | Economics, Microeconomics                                          | Applications will be accepted on a rolling basis.                                                                                                                                                                                     | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Staff Associate I                                                                                         | https://bit.ly/3AgUsw8      | Michael Best, Haoge Chang, Jonathan Dingel, Gautam Gowrisankaran, Sebastian Otero, Pietro Tebaldi, Tatiana Mocanu and Hugo Lhuillier                                                                                                                                                                                                                                                                                                                                   | The Program for Economic Research (PER), Columbia University                                                                                                                               | Development, Econometrics, Energy, Environmental, Health, Public Economics, Industrial Organization, International Economics, Labor, and Urban                                                                                                                       | Economics, Industrial Organization                                 | January 28, 2025                                                                                                                                                                                                                      | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Empirical Research Fellows                                                                                | https://bit.ly/40B7pMh      | Anna Tuchman and Caio Waisman                                                                                                                                                                                                                                                                                                                                                                                                                                          | Kellogg School of Management, Northwestern University                                                                                                                                      | Quantitative marketing, Applied microeconomics, policy analysis, machine learning                                                                                                                                                                                    | Economics, Microeconomics                                          | December 31, 2024 (full consideration)                                                                                                                                                                                                | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Fellow                                                                              | https://stanford.io/4hF4vMG | Daniel E. Ho                                                                                                                                                                                                                                                                                                                                                                                                                                                           | Stanford University, Regulation, Evaluation, & Governance Lab                                                                                                                              | Computer Science, Data Science, Statistics, Mathematics, Engineering, Economics, Public Policy                                                                                                                                                                       | Economics, Public Policy                                           | First round December 2, 2024, second round January 6, 2025                                                                                                                                                                            | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Assistant, Center on Opportunity and Social Mobility                                             | https://bit.ly/4feDBJB      | Kevin Corinth, Angela Rachidi, and Scott Winship                                                                                                                                                                                                                                                                                                                                                                                                                       | The American Enterprise Institute’s Center on Opportunity and Social Mobility (COSM)                                                                                                       | The research assistant will conduct extensive research on US safety-net programs, food insecurity, social capital, and intergenerational mobility, among other topics.                                                                                               | nan                                                                | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Fellow                                                                              | https://stanford.io/4foPOLA | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | Stanford King Center on Global Development                                                                                                                                                 | Development, Political Economy, Environmental, Data Science and Tech Innovation                                                                                                                                                                                      | nan                                                                | January 5, 2025                                                                                                                                                                                                                       | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Assistant, Economic Policy Studies                                                               | https://bit.ly/3AuNN1t      | Alex Brill, Kyle Pomerleau, Steven Kamin, and Sita Slavov                                                                                                                                                                                                                                                                                                                                                                                                              | The American Enterprise Institute (AEI)                                                                                                                                                    | Tax policy, public finance, and international macroeconomics                                                                                                                                                                                                         | Economics, Macroeconomics, Finance                                 | Rolling – position will start in summer 2025                                                                                                                                                                                          | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Fellows                                                                             | https://bit.ly/3Ydu2EL      | Ran Abramitzky, Lukas, Althoff, Adrien Bilal, Nicholas Bloom, Michael Boskin, Ignacio Cuesta, Omer Karaduman, Benny Kleinman, Rebecca Lester, Neale Mahoney, Melanie Morten, Amit Seru, Alessandra Voena, Shoshana Vasserman                                                                                                                                                                                                                                           | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Full-Time Research Assistant                                                                              | https://bit.ly/40m50oq      | Ellora Derenoncourt                                                                                                                                                                                                                                                                                                                                                                                                                                                    | National Bureau of Economic Research                                                                                                                                                       | Labor Economics                                                                                                                                                                                                                                                      | Economics                                                          | Rolling applicant reviews begin on October 22, 2024, and continue until the post is filled. Term: 6 month position                                                                                                                    | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Associate                                                                                        | https://bit.ly/4ejNxR7      | Manuel Adelino, Anna Cieslak, Nuno Clara, Paymon Khorrami, Arthur Taburet, Joao Thereze and Melanie Wallskog                                                                                                                                                                                                                                                                                                                                                           | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | January 31st                                                                                                                                                                                                                          | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | 2025 Full-Time Research Assistant                                                                         | https://bit.ly/4fgHzRD      | Professors Amir Kermani, Olivia Bordeu, Antoine Levy, and Laura Weiwu                                                                                                                                                                                                                                                                                                                                                                                                  | University of California, Berkeley                                                                                                                                                         | Urban economics, trade and spatial, labor economics, public finance, and applied microeconomics.                                                                                                                                                                     | Economics, Microeconomics, Finance                                 | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Staff Associate-Paul Milstein Center for Real Estate                                                      | https://bit.ly/3Ylg42J      | Paul Milstein Center for Real Estate                                                                                                                                                                                                                                                                                                                                                                                                                                   | Columbia Business School                                                                                                                                                                   | Finance                                                                                                                                                                                                                                                              | Finance                                                            | Rolling deadline                                                                                                                                                                                                                      | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Staff Associate-Finance                                                                                   | https://bit.ly/48gDYAV      | Finance Department                                                                                                                                                                                                                                                                                                                                                                                                                                                     | Columbia Business School                                                                                                                                                                   | Finance                                                                                                                                                                                                                                                              | Finance                                                            | Rolling deadline                                                                                                                                                                                                                      | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Staff Associate II                                                                                        | https://bit.ly/3YgEoCP      | Accounting Division                                                                                                                                                                                                                                                                                                                                                                                                                                                    | Columbia Business School                                                                                                                                                                   | Accounting Start Date: July 1, 2025                                                                                                                                                                                                                                  | nan                                                                | Applications will be reviewed on a rolling basis                                                                                                                                                                                      | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Professional-Development Innovation Lab                                                          | https://bit.ly/3zTNcq9      | Professor Michael Kremer                                                                                                                                                                                                                                                                                                                                                                                                                                               | The Development Innovation Lab (DIL) at the University of Chicago                                                                                                                          | Development economics                                                                                                                                                                                                                                                | Economics                                                          | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Specialist                                                                                       | https://bit.ly/3Yc0aY3      | Adam Chilton, Howard G. Krane Professor of Law and the Walter Mander Research Scholar                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | Applications reviewed on a rolling basis.                                                                                                                                                                                             | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Professional                                                                                     | https://bit.ly/3UcTGr4      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Specialist                                                                                       | https://bit.ly/4h5jQWh      | Jacob Goldin, Richard M. Lipton Professor of Tax Law                                                                                                                                                                                                                                                                                                                                                                                                                   | Coase-Sandor Institute for Law & Economics, University of Chicago Law School                                                                                                               | nan                                                                                                                                                                                                                                                                  | Economics                                                          | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Fellow in Environmental Economics                                                   | https://bit.ly/4eKVu2G      | Ishan Nath, Anna Russo, Wolfram Schlenker, Charles Taylor                                                                                                                                                                                                                                                                                                                                                                                                              | Harvard University                                                                                                                                                                         | Research areas will include environmental economics, geospatial data, agricultural economics, industrial organization, international trade, economic growth, and development.                                                                                        | Economics, Industrial Organization                                 | October 31st                                                                                                                                                                                                                          | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Fellow                                                                              | https://stanford.io/4fiHcpV | David Chan (moving from Stanford to Berkeley)                                                                                                                                                                                                                                                                                                                                                                                                                          | University of California, Berkeley                                                                                                                                                         | Health Economics, Applied Microeconomics                                                                                                                                                                                                                             | Economics, Microeconomics                                          | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Associate                                                                                        | https://bit.ly/3NqDOgv      | Jonathan Weigel and Augustin Bergeron                                                                                                                                                                                                                                                                                                                                                                                                                                  | UC Berkeley and Harvard University                                                                                                                                                         | Development Economics, Political Economy, Public Finance                                                                                                                                                                                                             | Economics, Finance                                                 | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Analyst                                                                                          | https://bit.ly/406B3Zo      | The Research Department                                                                                                                                                                                                                                                                                                                                                                                                                                                | Federal Reserve Bank of Cleveland                                                                                                                                                          | Economics, econometrics or statistics, and finance                                                                                                                                                                                                                   | Economics, Finance                                                 | Applications will be considered on a rolling basis, but interested individuals are strongly encouraged to apply no later than October 21, 2024.                                                                                       | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Empirical Research Fellows                                                                                | https://bit.ly/3Y5vmsc      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | Applied microeconomics, labor economics, industrial organization, innovation, organizational economics                                                                                                                                                               | Economics, Microeconomics, Industrial Organization                 | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Analyst under the Research Analyst Program (RAP)                                                 | https://bit.ly/4f2ipGs      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | International Monetary Fund                                                                                                                                                                | Economics, Finance, Statistics, Mathematics, Computer Science, Data Science                                                                                                                                                                                          | Economics, Finance                                                 | January 6, 2025                                                                                                                                                                                                                       | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Assistants                                                                                       | https://bit.ly/4dItRWr      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | Federal Reserve Bank of Dallas                                                                                                                                                             | Economics, statistics, econometrics                                                                                                                                                                                                                                  | Economics                                                          | Applications are reviewed on a rolling basis                                                                                                                                                                                          | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Full-Time Research Professionals (Pre-Doc)                                                                | https://bit.ly/401DgWa      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | Financial Economics                                                                                                                                                                                                                                                  | Economics                                                          | Applications will be evaluated on a rolling basis                                                                                                                                                                                     | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Professional                                                                                     | https://bit.ly/3NiWcId      | Professor Joshua Gottlieb (UChicago Harris School), Professor Matt Notowidigdo (Chicago Booth), Professor Zarek Brot-Goldberg (UChicago Harris School), and Professor Maggie Shi (UChicago Harris School)                                                                                                                                                                                                                                                              | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | Rolling Expected Start Date: July 1, 2025                                                                                                                                                                                             | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Analyst                                                                             | https://bit.ly/3Y6BwZc      | Judd Kessler and Corinne Low                                                                                                                                                                                                                                                                                                                                                                                                                                           | University of Pennsylvania (The Wharton School)                                                                                                                                            | Labor, Gender and Experimental Economics                                                                                                                                                                                                                             | Economics                                                          | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Analyst                                                                             | https://bit.ly/3Y6BwZc      | Ben Lockwood and Alex Rees-Jones                                                                                                                                                                                                                                                                                                                                                                                                                                       | University of Pennsylvania (The Wharton School)                                                                                                                                            | Public and Behavioral Economics                                                                                                                                                                                                                                      | Economics                                                          | Applications will be accepted on a rolling basis.                                                                                                                                                                                     | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Tobin Pre-Doctoral Fellowship                                                                             | https://bit.ly/3ZY6u8m      | Orazio Attanasio, Steven Berry, Cody Cook, Zack Cooper, Janet Currie, Eduardo Dávila, Ana De La O, John Eric Humphries, Zachary Liscow, Song Ma, Cormac O'Dea, Rohini Pande, Winnie van Dijk, Kevin Williams                                                                                                                                                                                                                                                           | Tobin Center and Department of Economics at Yale University                                                                                                                                | Applied Microeconomics, Development Economics, Economic Theory, Economics of Education, Health Economics, Financial Economics, Industrial Organization, Labor Economics, Law and Economics, Macroeconomics, Monetary Economics, Public Economics, Urban Economics    | Economics, Macroeconomics, Microeconomics, Industrial Organization | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Associate (Pre-doc)                                                                              | https://bit.ly/47UHvEW      | Erica Field                                                                                                                                                                                                                                                                                                                                                                                                                                                            | Duke Development Economics Lab (DevLab@Duke), Department of Economics at Duke University                                                                                                   | Applied experimental micro-economic development research; public policy                                                                                                                                                                                              | Economics, Public Policy                                           | The deadline is rolling, but hiring for summer 2025 with the specific date flexible for the selected candidate’s availability. Visa: Duke can provide visa sponsorship.                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | General Data Research Associate                                                                           | https://bit.ly/3TKyxnR      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | January 13, 2025 Early launch for General Data Research Associate  and General Field Research Associate positions: September 9 through October 4, 2024; Full launch opens for all positions: November 25, 2024                        | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Applied Machine Learning Pre-Doctoral Research Professional                                               | https://bit.ly/4dEBTjr      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | Center for Applied Artificial Intelligence at Chicago Booth                                                                                                                                | Applied Machine Learning; AI                                                                                                                                                                                                                                         | nan                                                                | October 11, 2024                                                                                                                                                                                                                      | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Associate                                                                                        | https://bit.ly/4eEfxiD      | Max Miller                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Harvard Business School                                                                                                                                                                    | Finance, macroeconomics, and political economy                                                                                                                                                                                                                       | Economics, Macroeconomics, Finance                                 | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Associate                                                                                        | https://bit.ly/4dvgw40      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | Harvard Kennedy School, The People Lab                                                                                                                                                     | Statistical analysis, anti-poverty, economics, public policy                                                                                                                                                                                                         | Economics, Public Policy                                           | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Coordinator                                                                                      | https://bit.ly/3XFgNLt      | Angela Duckworth & Katy Milkman                                                                                                                                                                                                                                                                                                                                                                                                                                        | Behavior Change for Good Initiative, The University of Pennsylvania                                                                                                                        | Behavioral Science, Education, Health, Statistics and Econometrics                                                                                                                                                                                                   | nan                                                                | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Empirical Research Fellowships (2025-2026)                                                                | https://stanford.io/2ZemVQw | John J. Donohue                                                                                                                                                                                                                                                                                                                                                                                                                                                        | Stanford Law School                                                                                                                                                                        | Crime Economics, Law and Economics, Public Policy                                                                                                                                                                                                                    | Economics, Public Policy                                           | Applications are reviewed on a rolling basis and accepted with priority until October 21st, 2024.                                                                                                                                     | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Professional                                                                                     | https://bit.ly/4gyBLED      | Luigi Zingales                                                                                                                                                                                                                                                                                                                                                                                                                                                         | The University of Chicago Booth School of Business, Stigler Center                                                                                                                         | Political Economy, Finance, Public Policy                                                                                                                                                                                                                            | Finance, Public Policy                                             | First review October 4, rolling thereafter                                                                                                                                                                                            | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Analyst                                                                                          | https://bit.ly/4ebW1uf      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Professional, Market Shaping Accelerator                                                         | https://bit.ly/3XhP4k0      | Michael Kremer                                                                                                                                                                                                                                                                                                                                                                                                                                                         | University of Chicago, Market Shaping Accelerator                                                                                                                                          | Microeconomics, economics of innovation, industrial organization, public policy,  pandemic preparedness, climate, market shaping                                                                                                                                     | Economics, Microeconomics, Industrial Organization, Public Policy  | First review, October 7; rolling thereafter                                                                                                                                                                                           | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Empirical Research Fellow                                                                                 | https://bit.ly/3ilyU5m      | David Matsa, Jacopo Ponticelli, Sean Higgins, Filippo Mezzanotti, Carola Frydman, etc.                                                                                                                                                                                                                                                                                                                                                                                 | Finance Department, Kellogg School of Management, Northwestern University                                                                                                                  | Finance                                                                                                                                                                                                                                                              | Finance                                                            | We will review applications on a rolling basis, with a target initial review on September 30, 2024.                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Bayesian Research Analyst                                                                                 | https://bit.ly/3XOeQ0C      | Dean Karlan                                                                                                                                                                                                                                                                                                                                                                                                                                                            | Global Poverty Research Lab, Northwestern University                                                                                                                                       | Development, Statistics and Econometrics, Bayesian Statistics, Meta-analysis                                                                                                                                                                                         | nan                                                                | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Associate                                                                                        | https://bit.ly/4dZPAu8      | Olivia Kim                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Entrepreneurial Management unit, Harvard Business School                                                                                                                                   | Projects related to how firms and households make financial decisions using detailed microeconomic administrative data and natural experiment-based empirical strategies.                                                                                            | nan                                                                | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Fellow                                                                              | https://bit.ly/4e5TKkk      | Marcella Alsan                                                                                                                                                                                                                                                                                                                                                                                                                                                         | Malcolm Wiener Center for Social Policy at Harvard Kennedy School                                                                                                                          | Health economics, Inequality                                                                                                                                                                                                                                         | Economics                                                          | Applications will be accepted and reviewed on a rolling basis.                                                                                                                                                                        | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Stanford GSB Research Fellows Program                                                                     | https://stanford.io/47lEUTZ | Rebecca Lester (Accounting), Juan Carlos Suarez Serrato (Economics), Ron Kasznik (Accounting), Maureen McNichols (Accounting), Jinwhan Kim (Accounting), Suzie Noh (Accounting), Rebecca Diamond (Economics), Claudia Allende Santa Cruz (Economics), Susan Athey (Economics), Antonio Coppola (Finance), Matteo Maggiori (Finance), Stefanos Zenios (Operations, Information & Technology), Julien Clement (Organizational Behavior), Andrew Hall (Political Economy) | Stanford Graduate School of Business                                                                                                                                                       | Accounting, Economics, Finance, Operations, Information & Technology, Organizational Behavior, Political Economy                                                                                                                                                     | Economics, Finance                                                 | October 18th, 2024. Applications submitted after the deadline will be considered on a rolling basis. View the details for all available dedicated projects here.                                                                      | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Fellow-University of Southern California                                            | https://bit.ly/47lDgSj      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | GIS Data Science Specialist                                                                               | https://bit.ly/3APOWR4      | Prof. Tamma Carleton                                                                                                                                                                                                                                                                                                                                                                                                                                                   | The Togo Data Lab, a joint initiative by the Togo’s Ministry of Digital Economy and Transformation (MoDET) and the Center for Effective Global Action (CEGA)                               | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Assistants                                                                                       | https://bit.ly/47y04yB      | PhD economists                                                                                                                                                                                                                                                                                                                                                                                                                                                         | The Board of Governors of the Federal Reserve System                                                                                                                                       | Microeconomic and macroeconomic theory; econometrics and statistics; data visualization; economic policy analysis; finance                                                                                                                                           | Finance                                                            | September 4- October 15 (deadline)                                                                                                                                                                                                    | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Associates                                                                                   | https://bit.ly/3TiTm9C      | Principal Investigators (PIs) at the Development Innovation Lab (DIL), including Professor Michael Kremer                                                                                                                                                                                                                                                                                                                                                              | The Development Innovation Lab at the University of Chicago                                                                                                                                | Development economics                                                                                                                                                                                                                                                | Economics                                                          | Priority deadline for this role is September 12th. Applications will be reviewed on a rolling basis. Current project locations are: Bhubaneshwar, Vijayawada, Delhi, Bangalore.                                                       | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Data Scientist                                                                                            | https://bit.ly/4dVNgV4      | The Data Science Team                                                                                                                                                                                                                                                                                                                                                                                                                                                  | Reddit                                                                                                                                                                                     | Quantitative or data science, machine learning, consumer-facing service/app operations, data safety etc.                                                                                                                                                             | nan                                                                | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | 2024 Research Analyst                                                                                     | https://bit.ly/3XzyvS3      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | International Monetary Fund                                                                                                                                                                | Economics, Finance, Statistics, Mathematics, Computer Science, Data Science                                                                                                                                                                                          | Economics, Finance                                                 | Rolling                                                                                                                                                                                                                               | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doctoral Research Fellow, Tech Ethics                                                                 | https://bit.ly/4cXq8Ej      | Yong Suk Lee , Program Chair in Technology Ethics for the Notre Dame Ethics Initiative and Associate Professor at the Keough School of Global Affairs                                                                                                                                                                                                                                                                                                                  | University of Notre Dame                                                                                                                                                                   | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Senior Research Assistant                                                                                 | https://bit.ly/4dQNaxE      | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | The Consumer Finance Institute (CFI), The Federal Reserve Bank of Philadelphia                                                                                                             | Research spans from urban economics to consumer finance to macroeconomics and monetary policy.                                                                                                                                                                       | Economics, Macroeconomics, Finance                                 | Rolling Learn more about the program.                                                                                                                                                                                                 | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Associate                                                                                        | https://bit.ly/3MveNAy      | Economic Research team                                                                                                                                                                                                                                                                                                                                                                                                                                                 | Federal Reserve Bank of San Francisco                                                                                                                                                      | Monetary policy, supervision, public information                                                                                                                                                                                                                     | nan                                                                | We will review applications on a rolling basis and keep the position open until all roles are filled.                                                                                                                                 | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Senior Research Specialists                                                                               | https://bit.ly/3ASueAf      | Eduardo Morales, Gene Grossman, and Stephen Redding                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | The first review date is October 1. Apply by October 15 to receive consideration.                                                                                                                                                     | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Pre-Doc (Summer 2025)                                                                                     | https://bit.ly/3z3B7xS      | Bill Evans                                                                                                                                                                                                                                                                                                                                                                                                                                                             | University of Notre Dame, Wilson Sheehan Lab for Economic Opportunities (LEO)                                                                                                              | Applied Microeconomics, Economics of Education, Health Economics, Labor Economics, Public Economics                                                                                                                                                                  | Economics, Microeconomics                                          | Sep 22, 2024 at 11:59 PM Eastern Time                                                                                                                                                                                                 | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| predoc   | Research Professional                                                                                     | https://bit.ly/3T5JO1G      | Manasi Deshpande, Magne Mogstad, and Evan Rose                                                                                                                                                                                                                                                                                                                                                                                                                         | Becker Friedman Institute for Economics (BFI), University of Chicago                                                                                                                       | Labor economics                                                                                                                                                                                                                                                      | Economics                                                          | September 30, 2024                                                                                                                                                                                                                    | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| nber     | nan                                                                                                       | nan                         | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | nan                                                                                                                                                                                                                                   | nan                                                                                    | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| ejm      | Faculty of Economics at Plaksha University, India                                                         | #ad-11235                   | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | Economics                                                          | 19 Nov 2024 30 Jul 2025 14 Nov 2025                                                                                                                                                                                                   | Plaksha University                                                                     | Various position types Assistant Professor • Associate Professor • Full Professor • Postdoctoral Scholar • Tenured Professor • Visiting Assistant Professor • Visiting Associate Professor • Visiting Professor • Research Assistant Various fields Development; Growth • Econometrics • Environmental; Ag. Econ. • Experimental Economics • Finance • International Finance/Macro • Macroeconomics; Monetary • Behavioral Economics • Any field • Business Economics • Marketing • Operations Research • Statistics • Microeconomic theory • Applied microeconomics | Various position types Assistant Professor • Associate Professor • Full Professor • Postdoctoral Scholar • Tenured Professor • Visiting Assistant Professor • Visiting Associate Professor • Visiting Professor • Research Assistant Various fields Development; Growth • Econometrics • Environmental; Ag. Econ. • Experimental Economics • Finance • International Finance/Macro • Macroeconomics; Monetary • Behavioral Economics • Any field • Business Economics • Marketing • Operations Research • Statistics • Microeconomic theory • Applied microeconomics |
| ejm      | Predoctoral Research Analyst -- Applied Microeconomics                                                    | #ad-10962                   | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | Economics, Microeconomics, Public Policy                           | 9 Oct 2024 1 Jul 2025 1 Jul 2025                                                                                                                                                                                                      | Business Economics and Public Policy, Wharton School University of Pennsylvania        | Research Assistant (Pre-Doc) Various fields Development; Growth • Environmental; Ag. Econ. • Experimental Economics • Labor; Demographic Economics • Public Economics • Behavioral Economics                                                                                                                                                                                                                                                                                                                                                                         | Research Assistant (Pre-Doc) Various fields Development; Growth • Environmental; Ag. Econ. • Experimental Economics • Labor; Demographic Economics • Public Economics • Behavioral Economics                                                                                                                                                                                                                                                                                                                                                                         |
| ejm      | Research Assistant                                                                                        | #ad-11355                   | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | Economics                                                          | 15 Dec 2024 15 Feb 2025 1 Apr 2025 1 Apr 2025                                                                                                                                                                                         | Animal Welfare Economics Working Group                                                 | Research Assistant Any field                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | Research Assistant Any field                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| ejm      | Research Assistant (CAGE)                                                                                 | #ad-11474                   | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | Economics                                                          | 13 Feb 2025 17 Mar 2025 17 Mar 2025                                                                                                                                                                                                   | Economics University of Warwick                                                        | Research Assistant Any field                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | Research Assistant Any field                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| ejm      | PhD scholarships in Economics and Management on Ports as Energy Transition Hubs                           | #ad-11423                   | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | Economics                                                          | 22 Jan 2025 17 Mar 2025 17 Mar 2025                                                                                                                                                                                                   | Department of Economics Copenhagen Business School                                     | Doctoral student Business Economics • Management, General                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | Doctoral student Business Economics • Management, General                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| ejm      | 1 Research Grant, IGIER Research Center                                                                   | #ad-11318                   | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | Economics                                                          | 16 Dec 2024 9 Mar 2025 9 Mar 2025                                                                                                                                                                                                     | Economics Bocconi University                                                           | Postdoctoral Scholar • Research Assistant Various fields Labor; Demographic Economics • Political Economy • Management, Information Technology                                                                                                                                                                                                                                                                                                                                                                                                                       | Postdoctoral Scholar • Research Assistant Various fields Labor; Demographic Economics • Political Economy • Management, Information Technology                                                                                                                                                                                                                                                                                                                                                                                                                       |
| ejm      | Post-Doctoral Associate in the Division of Social Science [Economics]                                     | #ad-11243                   | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | Economics                                                          | 16 Nov 2024 1 Dec 2024 1 Mar 2025 1 Mar 2025                                                                                                                                                                                          | Economics New York University Abu Dhabi                                                | Research Assistant Macroeconomics; Monetary                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | Research Assistant Macroeconomics; Monetary                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| ejm      | Research Assistant                                                                                        | #ad-11411                   | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | nan                                                                | 15 Jan 2025 1 Mar 2025 1 Mar 2025                                                                                                                                                                                                     | FutureTech Massachusetts Institute of Technology                                       | Research Assistant • Research Assistant (Pre-Doc) Econometrics                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | Research Assistant • Research Assistant (Pre-Doc) Econometrics                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| ejm      | Research Engineer or Postdoctoral position in Health Economics                                            | #ad-10723                   | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | Economics                                                          | 13 Aug 2024 15 Sep 2024 28 Feb 2025 28 Feb 2025                                                                                                                                                                                       | Information Systems, Decision Sciences and Statistics Department ESSEC Business School | Postdoctoral Scholar • Other academic • Research Assistant Econometrics • Health; Education; Welfare                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | Postdoctoral Scholar • Other academic • Research Assistant Econometrics • Health; Education; Welfare                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| ejm      | Several postdoctoral researchers in microeconomic theory                                                  | #ad-11107                   | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | Economics                                                          | 5 Nov 2024 28 Feb 2025 28 Feb 2025                                                                                                                                                                                                    | Department of Economics University of Bonn                                             | Postdoctoral Scholar • Research Assistant Microeconomic theory                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | Postdoctoral Scholar • Research Assistant Microeconomic theory                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| ejm      | Predoctoral Research Associate in Economics                                                               | #ad-11441                   | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | Economics                                                          | 28 Jan 2025 27 Feb 2025 28 Feb 2025                                                                                                                                                                                                   | Department of Economics University of Zurich                                           | Research Assistant (Pre-Doc) Any field                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | Research Assistant (Pre-Doc) Any field                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| ejm      | PhD Student in Economics - Teaching and Research Associate                                                | #ad-11446                   | nan                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | nan                                                                                                                                                                                        | nan                                                                                                                                                                                                                                                                  | Economics                                                          | 29 Jan 2025 19 Feb 2025 19 Feb 2025                                                                                                                                                                                                   | Economics Vienna University of Economics and Business                                  | Research Assistant (Pre-Doc) Any field                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | Research Assistant (Pre-Doc) Any field                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |